### Setting the CheXpert dataset labels to "U-Zeroes" mode; setting all "uncertain" labels to zeroes
keeping only the relevant columns with reordering the CheXpert competition labels to be the first five label columns, with creating csv label files of all 14 labels and csv label files of only the 5 CheXpert competition labels.

__Reference__: The 'Leveraging Uncertainty Labels' section in the official CheXpert website: [link](https://stanfordmlgroup.github.io/competitions/chexpert/)

In [1]:
import pandas as pd

### 1- Reading train and validation labels csv files

In [2]:
train_labels = pd.read_csv("../../labels/train.csv")

In [3]:
train_labels.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [4]:
train_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,NaN,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,-1.0,0.0,1.0,NaN,NaN,NaN
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,0.0,-1.0,NaN,-1.0,NaN,NaN,NaN
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,Female,0,Frontal,AP,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,Female,0,Frontal,AP,NaN,NaN,1.0,1.0,NaN,NaN,NaN,-1.0,1.0,0.0,NaN,NaN,NaN,0.0


In [5]:
validation_labels = pd.read_csv("../../labels/valid.csv")

In [6]:
validation_labels.columns

Index(['Path', 'Sex', 'Age', 'Frontal/Lateral', 'AP/PA', 'No Finding',
       'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity',
       'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis',
       'Pneumothorax', 'Pleural Effusion', 'Pleural Other', 'Fracture',
       'Support Devices'],
      dtype='object')

In [7]:
validation_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,Male,70,Lateral,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,Female,57,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,Male,65,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,Male,71,Frontal,AP,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,Female,45,Frontal,AP,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2- Checking how many Front and Lateral pictures exist

In [8]:
count_frontal_train = train_labels["Frontal/Lateral"].str.contains("Frontal", na=False).sum()
count_frontal_train

191027

In [9]:
count_frontal_train / len(train_labels)

0.8550359422417575

In [10]:
count_lateral_train = train_labels["Frontal/Lateral"].str.contains("Lateral", na=False).sum()
count_lateral_train

32387

In [11]:
count_lateral_train / len(train_labels)

0.14496405775824253

In [12]:
count_frontal_validation = validation_labels["Frontal/Lateral"].str.contains("Frontal", na=False).sum()
count_frontal_validation

202

In [13]:
count_frontal_validation / len(validation_labels)

0.8632478632478633

In [14]:
count_lateral_validation = validation_labels["Frontal/Lateral"].str.contains("Lateral", na=False).sum()
count_lateral_validation

32

In [15]:
count_lateral_validation / len(validation_labels)

0.13675213675213677

### 3- Replacing NaNs with zeroes, and replacing the 'undecided (-1) labels with zeroes'

In [16]:
train_labels = train_labels.fillna(0)
validation_labels = validation_labels.fillna(0)

In [17]:
train_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,0.0,0.0,-1.0,1.0,0.0,-1.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,1.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,0,0.0,0.0,0.0,1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,Male,59,Frontal,AP,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,0.0,0.0
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,Male,59,Frontal,AP,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,Female,0,Frontal,AP,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,Female,0,Frontal,AP,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
validation_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,Male,70,Lateral,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,Female,57,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,Male,65,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,Male,71,Frontal,AP,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,Female,45,Frontal,AP,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
train_labels = train_labels.replace(-1, 0)

In [20]:
train_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/train/patient00001/study1/view1_...,Female,68,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,Female,87,Frontal,AP,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,Female,83,Frontal,AP,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,Female,83,Lateral,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,Male,41,Frontal,AP,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,Male,59,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,Male,59,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,Female,0,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,Female,0,Frontal,AP,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [21]:
validation_labels = validation_labels.replace(-1, 0)

In [22]:
validation_labels

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,Male,73,Frontal,AP,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,Male,70,Frontal,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,Male,70,Lateral,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,Male,85,Frontal,AP,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,Female,42,Frontal,AP,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,Female,57,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,Male,65,Frontal,AP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,Male,71,Frontal,AP,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,Female,45,Frontal,AP,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 4- Reordering the columns so that the CheXpert competition labels: (Atelectasis, Cardiomegaly, Consolidation, Edema, Pleural Effusion) would be the first five label columns (for easier extraction of relevant model outputs) and removing irrelevant columns

In [23]:
reordered_columns = [
    "Path",
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Pleural Effusion",
    "Pleural Other",
    "Pneumonia",
    "Pneumothorax",
    "Enlarged Cardiomediastinum",
    "Lung Opacity",
    "Lung Lesion",
    "Fracture",
    "Support Devices",
    "No Finding"
]

In [24]:
train_labels = train_labels[reordered_columns]

In [25]:
train_labels

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Enlarged Cardiomediastinum,Lung Opacity,Lung Lesion,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [26]:
validation_labels = validation_labels[reordered_columns]

In [27]:
validation_labels

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Enlarged Cardiomediastinum,Lung Opacity,Lung Lesion,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


### 5- Checking distribution of labels

#### 5.1 Atelectasis

In [28]:
train_labels["Atelectasis"].sum()

33376.0

In [29]:
train_labels["Atelectasis"].sum() / len(train_labels)

0.14939081704817067

In [30]:
validation_labels["Atelectasis"].sum()

80.0

In [31]:
validation_labels["Atelectasis"].sum() / len(validation_labels)

0.3418803418803419

#### 5.2 Cardiomegaly

In [32]:
train_labels["Cardiomegaly"].sum()

27000.0

In [33]:
train_labels["Cardiomegaly"].sum() / len(train_labels)

0.12085187141360881

In [34]:
validation_labels["Cardiomegaly"].sum()

68.0

In [35]:
validation_labels["Cardiomegaly"].sum() / len(validation_labels)

0.2905982905982906

#### 5.3 Consolidation

In [36]:
train_labels["Consolidation"].sum()

14783.0

In [37]:
train_labels["Consolidation"].sum() / len(train_labels)

0.0661686375965696

In [38]:
validation_labels["Consolidation"].sum()

33.0

In [39]:
validation_labels["Consolidation"].sum() / len(validation_labels)

0.14102564102564102

#### 5.4 Edema 

In [40]:
train_labels["Edema"].sum()

52246.0

In [41]:
train_labels["Edema"].sum() / len(train_labels)

0.2338528471805706

In [42]:
validation_labels["Edema"].sum()

45.0

In [43]:
validation_labels["Edema"].sum() / len(validation_labels)

0.19230769230769232

#### 5.5 Pleural Effusion

In [44]:
train_labels["Pleural Effusion"].sum()

86187.0

In [45]:
train_labels["Pleural Effusion"].sum() / len(train_labels)

0.38577260153795195

In [46]:
validation_labels["Pleural Effusion"].sum()

67.0

In [47]:
validation_labels["Pleural Effusion"].sum() / len(validation_labels)

0.2863247863247863

#### 5.6 Pleural Other

In [48]:
train_labels["Pleural Other"].sum()

3523.0

In [49]:
train_labels["Pleural Other"].sum() / len(train_labels)

0.01576893122185718

In [50]:
validation_labels["Pleural Other"].sum()

1.0

In [51]:
validation_labels["Pleural Other"].sum() / len(validation_labels)

0.004273504273504274

#### 5.7 Pneumonia

In [52]:
train_labels["Pneumonia"].sum()

6039.0

In [53]:
train_labels["Pneumonia"].sum() / len(train_labels)

0.027030535239510507

In [54]:
validation_labels["Pneumonia"].sum()

8.0

In [55]:
validation_labels["Pneumonia"].sum() / len(validation_labels)

0.03418803418803419

#### 5.8 Pneumothorax

In [56]:
train_labels["Pneumothorax"].sum()

19448.0

In [57]:
train_labels["Pneumothorax"].sum() / len(train_labels)

0.08704915537969868

In [58]:
validation_labels["Pneumothorax"].sum()

8.0

In [59]:
validation_labels["Pneumothorax"].sum() / len(validation_labels)

0.03418803418803419

#### 5.9 Enlarged Cardiomediastinum

In [60]:
train_labels["Enlarged Cardiomediastinum"].sum()

10798.0

In [61]:
train_labels["Enlarged Cardiomediastinum"].sum() / len(train_labels)

0.048331796574968444

In [62]:
validation_labels["Enlarged Cardiomediastinum"].sum()

109.0

In [63]:
validation_labels["Enlarged Cardiomediastinum"].sum() / len(validation_labels)

0.4658119658119658

#### 5.10 Lung Opacity

In [64]:
train_labels["Lung Opacity"].sum()

105581.0

In [65]:
train_labels["Lung Opacity"].sum() / len(train_labels)

0.47258005317482343

In [66]:
validation_labels["Lung Opacity"].sum()

126.0

In [67]:
validation_labels["Lung Opacity"].sum() / len(validation_labels)

0.5384615384615384

#### 5.11 Lung Lesion

In [68]:
train_labels["Lung Lesion"].sum()

9186.0

In [69]:
train_labels["Lung Lesion"].sum() / len(train_labels)

0.041116492252052245

In [70]:
validation_labels["Lung Lesion"].sum()

1.0

In [71]:
validation_labels["Lung Lesion"].sum() / len(validation_labels)

0.004273504273504274

#### 5.12 Fracture

In [72]:
train_labels["Fracture"].sum()

9040.0

In [73]:
train_labels["Fracture"].sum() / len(train_labels)

0.040462996947371245

In [74]:
validation_labels["Fracture"].sum()

0.0

In [75]:
validation_labels["Fracture"].sum() / len(validation_labels)

0.0

#### 5.13 Support Devices

In [76]:
train_labels["Support Devices"].sum()

116001.0

In [77]:
train_labels["Support Devices"].sum() / len(train_labels)

0.5192199235500013

In [78]:
validation_labels["Support Devices"].sum()

107.0

In [79]:
validation_labels["Support Devices"].sum() / len(validation_labels)

0.45726495726495725

#### 5.14 No Finding

In [80]:
train_labels["No Finding"].sum()

22381.0

In [81]:
train_labels["No Finding"].sum() / len(train_labels)

0.10017724941140663

In [82]:
validation_labels["No Finding"].sum()

38.0

In [83]:
validation_labels["No Finding"].sum() / len(validation_labels)

0.1623931623931624

### 6- Saving processed labels

In [84]:
train_labels

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Enlarged Cardiomediastinum,Lung Opacity,Lung Lesion,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/train/patient00001/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [85]:
train_labels.to_csv("../../labels/train_u-zeroes.csv")

In [86]:
validation_labels

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Enlarged Cardiomediastinum,Lung Opacity,Lung Lesion,Fracture,Support Devices,No Finding
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [87]:
validation_labels.to_csv("../../labels/validation_u-zeroes.csv")

### 7- Saving CheXpert labels only version of the labels

In [88]:
chexpert_labels = [
    "Path",
    "Atelectasis",
    "Cardiomegaly",
    "Consolidation",
    "Edema",
    "Pleural Effusion"
]

In [89]:
train_labels_chexpert = train_labels[chexpert_labels]

In [90]:
train_labels_chexpert

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
0,CheXpert-v1.0/train/patient00001/study1/view1_...,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0/train/patient00002/study2/view1_...,0.0,0.0,0.0,0.0,0.0
2,CheXpert-v1.0/train/patient00002/study1/view1_...,0.0,0.0,0.0,0.0,0.0
3,CheXpert-v1.0/train/patient00002/study1/view2_...,0.0,0.0,0.0,0.0,0.0
4,CheXpert-v1.0/train/patient00003/study1/view1_...,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
223409,CheXpert-v1.0/train/patient64537/study2/view1_...,0.0,0.0,0.0,0.0,1.0
223410,CheXpert-v1.0/train/patient64537/study1/view1_...,0.0,0.0,0.0,0.0,0.0
223411,CheXpert-v1.0/train/patient64538/study1/view1_...,0.0,0.0,0.0,0.0,0.0
223412,CheXpert-v1.0/train/patient64539/study1/view1_...,1.0,1.0,0.0,0.0,0.0


In [91]:
train_labels_chexpert.to_csv("../../labels/train_u-zeroes_chexpert_labels.csv")

In [92]:
validation_labels_chexpert = validation_labels[chexpert_labels]

In [93]:
validation_labels_chexpert

,Path,Atelectasis,Cardiomegaly,Consolidation,Edema,Pleural Effusion
0,CheXpert-v1.0/valid/patient64541/study1/view1_...,0.0,1.0,0.0,0.0,0.0
1,CheXpert-v1.0/valid/patient64542/study1/view1_...,0.0,0.0,0.0,0.0,0.0
2,CheXpert-v1.0/valid/patient64542/study1/view2_...,0.0,0.0,0.0,0.0,0.0
3,CheXpert-v1.0/valid/patient64543/study1/view1_...,0.0,0.0,0.0,1.0,0.0
4,CheXpert-v1.0/valid/patient64544/study1/view1_...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
229,CheXpert-v1.0/valid/patient64736/study1/view1_...,0.0,0.0,0.0,0.0,0.0
230,CheXpert-v1.0/valid/patient64737/study1/view1_...,0.0,0.0,0.0,0.0,0.0
231,CheXpert-v1.0/valid/patient64738/study1/view1_...,0.0,1.0,0.0,1.0,0.0
232,CheXpert-v1.0/valid/patient64739/study1/view1_...,0.0,0.0,0.0,0.0,0.0


In [94]:
validation_labels_chexpert.to_csv("../../labels/validation_u-zeroes_chexpert_labels.csv")